# Heatwave Training 01 
+ Train on images to predict Heatwave  
+ add day of the year to input

## Imports and Initial Variables

In [1]:
from hw_training_prep import load_images, normalize_images, build_CNN_model, download_dataset_of_images
from hw_training_01 import Heatwave_Model_Training, LayerDetails

import numpy as np

from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import Workspace, Datastore, Dataset, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset

from matplotlib import pyplot as plt
%matplotlib inline


ws : Workspace.create(name='hes', subscription_id='0e150cbb-ad2f-47a1-849c-c5d0527afd2b', resource_group='hes-nasa-msft')
all imports successful
tf version 2.1.0


In [2]:
ws = Workspace.from_config()
print('workspace:\n', ws)
sub_folder_name = 'aoi_107_2021_11_11'
datastore_name = 'workspaceblobstore'

workspace:
 Workspace.create(name='hes', subscription_id='0e150cbb-ad2f-47a1-849c-c5d0527afd2b', resource_group='hes-nasa-msft')


## Download images from Dataset -- if the parent folder does not already exist locally

In [3]:
dataset_prefix = 'aoi_107_2021_11_11'
download_dataset_of_images(dataset_prefix)

The folder aoi_107_2021_11_11 already exists. Will not proceed with the download.


## Load images using Heatwave Model Training class

In [4]:
hmt = Heatwave_Model_Training(images_folder_name=dataset_prefix)
hmt.load_subsets_shuffle_and_normalize(limit=0)
hmt.print_dataset_summary()

label folder aoi_107_2021_11_11/train/0
label folder aoi_107_2021_11_11/train/1
label folder aoi_107_2021_11_11/validate/0
label folder aoi_107_2021_11_11/validate/1
label folder aoi_107_2021_11_11/test/0
label folder aoi_107_2021_11_11/test/1
Count of images =  6568
Label 0 -- count 4655
Label 1 -- count 1913
len(train_X) 6568 len(train_Y) 6568 len(train_X_names) 6568
(231, 349, 3) 1 Img_hw_area_pct_30__1983__d_228__var_tasmax.png
(231, 349, 3) 0 Img_hw_area_pct_30__2029__d_250__var_tasmax.png

len(validate_X) 1406
len(test_X) 1410


## Prepare a CNN model using Heatwave Model Training, LayerDetails classes

In [5]:
print('Preparing model for training 01 -- images only...')

# layers
build_with_layers = []
build_with_layers.append(LayerDetails(layerType='C', filters=16, kernel_size= (3,3), pool_size=None, strides=None))
build_with_layers.append(LayerDetails(layerType='M', filters=None, kernel_size= None, pool_size=(2,2), strides=(2,2)))
build_with_layers.append(LayerDetails(layerType='C', filters=32, kernel_size= (3,3), pool_size=None, strides=None))               

model = hmt.prepare_model(build_with_layers)
learning_rate = 0.01
epochs = 35
patience=5
print(model, '\n')

hmt.train_binary_classification_model(model, learning_rate, epochs, patience)

Preparing model for training 01 -- images only...
Preparing model for training...
Creating model...
input_shape = [231, 349, 3] num_classes = 1
Model: "model_1639037569"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 231, 349, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 231, 349, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 115, 174, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 115, 174, 32)      4640      
_________________________________________________________________
flatten (Flatten)            (None, 640320)            0         
_________________________________________________________________
dense (Dense)                (None, 1)